# $\color{orange}{\text{Multiway Split Sparse Decision Tree}}$


<span style='color:orange'> 

* <a href="#Introduction">Introduction</a> <br>
* <a href="#Bit-Maths">Bit Maths</a> <br>
* <a href="#Design-and-analysis-of-Algorithms">Design and analysis of Algorithms</a> <br>
* <a href="#Implementation">Implementation</a> <br>
 - <a href="#Refactoring">Refactoring</a> <br>
 - <a href="#Unit-Tests">Unit Tests</a> <br>
 - <a href="#Static-Analysis">Static Analysis</a> <br>
 - <a href="#Dynamic-Code-Analysis">Dynamic Code Analysis</a> <br>
 - <a href="#Benchmarking">Benchmarking</a> <br>
* <a href="#Embedding">Embedding MGOSDT in Python</a> <br>
* <a href="#Contribution">Contribution</a> <br>
* <a href="#Future">Future work</a> <br>
</span>

## Introduction
<img src="tree0.png" alt="drawing" width="550"/>
Decision tree has been popular since the early days of Machine Learning due to its interpretibility. Historically, the way we construct it is simple. We start with some root, keep branching it out til we hit some kind of stopping condition, at which point we can optionally prune some of the branches to avoid overfitting. The problem with this very plain algorithm (top down, prune it afterward) is the lack of optimality, if we choose the wrong point at the very beginning, there is no way to undo it. There has been numerous attempts to fix this problem via Mathematical optimization solver (or neural networks). However, in order to fully optimize a decision tree, we need to go through a search space that is both in both theory and practice hard. It does combinotorially explosion in the number of subtrees one can consider. 

The goal of my work is to produce a optimal sparse, **multiway-split** decision tree.  We follow a specialized algorithm that combines *dynamic programming* and *branch and bound* to optimize a generalize objective:
$$ \min_{\text{tree}} \hat{L}(\text{tree}, \{(x_i, y_i)\}_i) = l(\text{tree}, \{(x_i, y_i)\}_i) + \lambda(\#\text{leaves in tree}).$$
We minimize the misclassification error over all possible n-ary trees and sparsity. Here $\lambda$ is the trade-off parameter that govern the predictive performance and sparsity. (or in brief, regularizing the number of leaves).

We remark that this algorithm solve the problem of Optimality (NP hard) by leveraging computational caching. No greedy splitting and pruning like C4.5 and CART. When we could solve the problem of optimality, we get sparse accurate tree. Our approach has several important insights

1. **Analytical Bounds**: The collection of bounds show that some partial trees can never be extended to form optimal tree, thus reducing the search space, without sacrificing the optimality of our algorithm)

2. Dynamic Programming and **dependency graph** (See pptx)

* Start with some datasets, apply some naive labels
* Split into subsets using each feature
* Keep splitting until higher accuracy is attained
* Consolidate any duplication is found. (Solution for one duplicated instance can be used as the solution by another instance.
* The DP formulation creates a dependency graph between sets and subsets. Each set is responsible for finding optimal features to sub-divide itself into additional subsets. Then each subset decide the best fit to split. Once enough subsets are decided, we collapse the trees,  the optimal tree emerge as a Directed Acyclic graph of best features.

3. Tree representation by its leaves (Store bounds and intermediate results within each leave) ![tree representation](leaves.png)

4. **Permutation map**: Discover identical trees already evaluated.
5. **Leaf-based representation**: We store a bit vector, indicating which data point has features corresponding to the features described by its leaves (**Bit masking**. See `bitmask.cpp`, `dataset.cpp` and `encoder.cpp`)

```cpp
#include <iostream>
#include <cassert>
#include <vector>
// The purpose of the bit function is to tell us whether a certain bit of an int is set to 1
// By doing &1 we are getting a value where all bits other than bit 0 are 0
// The bit function does
// 1 move the desired bit to bit 0 (by shifting)
// 2 set all bits other than bit 0 to 0 (by adding)
// 3 result true if the result is not 0 (by converting to bool)
static bool bit(int index, int value)
{
   return value & (1 << index); 
}

#include <vector>
std::vector<bool> makeBitVector(size_t rain, bool construction, bool rush_hour, bool friday)
{
  std::vector<bool> result(5, false);
  result[0] = bit(0, rain);// result[0] is the first bit of rain (bit 0)
  result[1] = bit(1, rain);// result[1] is the 2nd bit of rain (bit 1)
  result[2] = construction;
  result[3] = rush_hour;
  result[4] = friday;
  return result;
}
static bool bit(int index, int value)
{
   return value & (1 << index); // check if index^th bit is present in the subset
}
```
[bitmasking via vector of bool](https://godbolt.org/z/qYnxE4rzx)

[bitmasking via bitset](https://godbolt.org/z/WPbf7GKhc)
[bitmasking multiple bits](https://godbolt.org/z/456ovEcx8)

6. Caching of intermediate results make our computations very fast. (Since cache memory is the fastest type of memory in your computer)

Question: Does each set correspond to a task? or do the sets and subsets have nothing to do with the arrangement of work?

In my [codes](https://gitlab.com/leannejdong/mgosdt/-/blob/main/src/optimizer/extraction/models.hpp)
the function Line25 creates a set of optimal trees. It is essentially the dependency graph in the context of concurrency programming. Indeed, tree, subtree; problem, sub-problem; set, subset in mgosdt are the `task`, which are the problems entered by the bit vectors (from the bitmask class).

7. Incremental computation
The bounds of bit vector in each leave also let us to use incremental computations to evaluate the children of the leaves- should we decide to split next or not...
5. Multiway split

* Give a more natural way to handle multivalued categorical features than binary splits. Even if we have 3 categories, a 2-way split still seems less interpretable

* Give a more interpretable DT algorithm that can handle a larger variety of datasets.

* m-way tree works much nicer on GPUs than binary trees. The shallowness usually help in the sense that we could compress it and save memory storing information. For instance,  B-tree, a kind of balanced search tree that is optimized for external memory. This is due to its shallow structure, so the height of a B tree is minimized, meaning that we don't have to access all the items everytime we call the method recursively.

### Binary search tree vs Binary decision tree

A **binary search tree** is an efficient data structure for storing information which you’d like to look up later. For example, you can store n integers and look one up in log n time. At each node in the binary search tree you are simply asking whether the number you’re looking for is higher or lower (hence binary), until eventually you find what you’re looking for (think binary search).

A **binary decision tree**, at least in the context of machine learning, is a function that maps an input space of data to an output space of classes. For example, say you want classify whether a product will get sold out on Black Friday or not. The input space is all the information about the product such as historical sales, product type, amount discounted, number in stock, etc. The output space has two classes: will sell out, or won’t sell out.

Each node in the binary decision tree asks a binary question about the data, e.g does the product have a high discount?, does the product usually sell well?. Based on the answer to each question you will take the left or right branch to the next node, following it down to the bottom (a leaf node) where you will find your final answer about whether it will sell out or not.

More generally, you can think of a binary decision tree as a decision making tool. It asks you a series of questions and gives you a decision based on your answers.

### Examples of multiway-split trees 

B* trees, suffix trees

### Represent each subproblem by bit vector

```cpp
// a program converts integers to their bit representation 
#include <iostream>
#include <bitset>
int main(){
    for(unsigned i; std::cin>>i;)
        std::cout << std::dec << i << "=="
                  << std::hex << "0x" << i << "=="
                  << std::bitset<8*sizeof(unsigned)>{i}<<"\n";//  #bits in an unsigned int
// unsigned types work better with bit operations since we typically don't care about 
// negative numbers when dealing with bitwise operations. 
// Using an unsigned type gives us one more bit to work with
}
```

## Bit Maths
Information gets stored in computer in term of binary numbers.

**Bitmasks** are a very useful way to compress multiple boolean flags in a single variable. It can reduce memory usage and operations on bits are basically as fast as they can get. In practice, any time you want to have multiple flags describing something in your application, a bitmask could be the right tool for the job. 

Mask in Bitmask means hiding something. And Bitmask is nothing but a binary number that represents something. Take an example. Consider the set $A = \{1, 2, 3, 4, 5\}$. How do we represent the subset $\{2, 4\}$ using a bitmask of length 5? (**Answer**: the bit mask 01010 represents the subset $\{2, 4\}$). The benefits of using bitmask are

* Set the $i^{th}$ bit using bitwise or: $b|(1<< i)$. Take $i = 0$.
* Unset(clear) the $i^{th}$ bit: $b\&!(1<< i)$. Take $i = 1$.
* Set the $i^{th}$ bit: $b\&(1<< i)$. Take $i = 3$. Then
$$(1<<i) = 01000$$
$$01010\& 01000 = 01000$$
<font color = orange>Exercise</font>: Given a set, count how many subsets have sum of elements greater than or equal to a given value. [subset sum](https://godbolt.org/z/1YdcrTdoK)

We start by creating a class called Bitmask. This class will handle all of our bit manipulation.

In [1]:
class Bitmask
{    
    public:
        Bitmask();

        // Overwrites this bitmask.
        void SetMask(Bitmask& other); 

        // Returns binary representation of bitmask.
        uint32_t GetMask() const; 

        // Returns true if bit at pos = 1, else false.
        bool GetBit(int pos) const; 

        // Sets bit at specified pos to 1 or 0 (true or false).
        void SetBit(int pos, bool on);

        // Sets bit at pos to 1.
        void SetBit(int pos); 

        // Sets bit at pos to 0.
        void ClearBit(int pos);

        // Sets all bits to 0.
        void Clear(); 

    private:
        uint32_t bits; // 1.
};

```cpp
Bitmask::Bitmask() : bits(0) { }

void Bitmask::SetMask(Bitmask& other)
{
    bits = other.GetMask();
}

uint32_t Bitmask::GetMask() const
{
    return bits;
}

bool Bitmask::GetBit(int pos) const
{
    return (bits & (1 << pos)) != 0; // 1
}

// A simple helper method that calls set or clear bit
void Bitmask::SetBit(int pos, bool on)
{
    if(on)
    {
        SetBit(pos);
    }
    else
    {
        ClearBit(pos);
    }
}

void Bitmask::SetBit(int pos)
{
    bits = bits | 1 << pos; // 2
}

void Bitmask::ClearBit(int pos)
{
    bits = bits & ~(1 << pos); // 3
}

void Bitmask::Clear()
{
    bits = 0;
}
```

In MGOSDT, We define a function module as a collection of functions. 
* This declaration acts as both a function module and a container class. 

* The static class methods implements a function module providing operations on arrays of type bitblock, which can be allocated on the stack.

* The non-static class methods implements a heap-allocated equivalence, which supports the same methods @note: Many of the binary operations assume that operands have the same length

[bitmask.hpp](https://gitlab.com/leannejdong/mgosdt/-/blob/dev1/src/bitmask.hpp)
[bitmask.cpp](https://gitlab.com/leannejdong/mgosdt/-/blob/dev1/src/bitmask.cpp)


## Design and analysis of Algorithms

### The DPB algorithm

* Operates on weighted, additive, non-negative loss functions.

$$ \min_{\text{tree}} \hat{L}(\text{tree}, \{(x_i, y_i)\}_i) = l(\text{tree}, \{(x_i, y_i)\}_i) + \lambda(\#\text{leaves in tree}).$$

* **DP** allows us to decompose a problem into smaller child problems that can be solved recursively through a function call.

* The **parallelism** allows us to solve the sub problems in parallel by delegating work to a separate thread.

* The **branch and bound** algorithm let us to prune the search space. In the case of mgosdt, the analytical bounds allows us to eliminate some part of the search space.
How does BB help? If we know the optimal cost is less than X, and we know the current branch of our search space cost more than X, then we can disregard this branch, since we know it won't lead to the optimal cost.

### The Primary Data structure:

GOSDT mains two primary data structures:

* a concurrent *priority queue* is used to schedule problems to solve. Recall, a priority queue maintains an ordering in the queue based on the priorities of individual queue items. A normal queue has a FIFO policy, whereas a priorrity queue sorts its items. The C++ stl implementation `std::priority_queue` is not thread-safe. The *concurrent* usage means that the values from all three can change due to push/pop methods in other threads. The `tbb::concurrent_priority_queue` support `std::priority_queue`'s methods of size, empty and swap and is **thread-safe**.

* a dependency graph is used to store problems and their dependency relationships. The dependency relationship $dep(p_{\pi}, p_c)$ is defined between problems $p_{\pi}$ and $p_c$ only if the solution of $p_{\pi}$ depends on the solution of $p_c$. Each $p_c$ is further specified as $p^j_l$ or $p^j_r$ indicating that it is the left and right branch produced by splitting on feature $j$.

### Algorithms

* Algorithm 1
![GOSDT](GOSDT.png)
* Algorithm 2 get_lower_bound$(s, Z, z^-, z^+)->lb)$
* Algorithm 3 get_upper_bound$(s, Z, z^-, z^+)->ub)$
* Algorithm 4 fails_bound$(p)->v$
* Algorithm 5 $\text{split}(s, j, Z)->s_l, s_r$
![split](split.png)
* Algorithm 6 an extraction algorithm that is used to construct the optimal tree from the dependency graph once the main GOSDT algorithm completes.
![extract](extract.png)


In my [codes](https://gitlab.com/leannejdong/mgosdt/-/blob/main/src/optimizer/extraction/models.hpp)
the function Line25 creates a set of optimal trees. It is essentially the dependency graph in the context of concurrency programming. Indeed, tree, subtree; problem, sub-problem; set, subset in mgosdt are the `task`, which are the problems entered by the bit vectors (from the bitmask class).

## Dynamic programming formulation

* Each dataset presents a classfication problem.
* When we alter the data by condition, the resulting subset presents a new problem.
* Each filtering creates a new subproblem.
* If two sets of conditions result in the same subset, then the solution of that subset can be used for both sets of conditions.

This allows us to reduce the amount of computations to reach the solutions.

## Contribution

Our work presents

* A new algorithm convert binary decision tree to n-ary decision tree.
* Efficient memory management leads to a speed up! Prevent memory allocation does generally speed up a program.
* Performance ptimization. 
 - Completed profiling that allows us to detect which part of mgosdt are slow.
 - We came up with ideas about what speed up mgosdt
 - We experimented and performed benchmarking which validated things actually work.

## Some common questions on multiway vs Binary decision tree?

Given the tree's structure, any one of the `N` attributes can be encoded in $log_2(N)` bits.
* Do n-ary DT lead a run time improvement?

No. There is no difference in term of complexity (both time and space). Every node may have multiple children (over 2) but the running time is still $O(\log N)$.
$$\log_n N = \frac{\log_a N}{\log_a b}.$$
Here, the $\log_a b$ is just a constant so it does not matter. So we can change the base of the logarithm and the running time complexity for the algorithm stay the same.
$$ O(c*\log N) = c O(\log N) = O(\log N) .$$ This is why, the branching factor (i.e. the number of children a node may have) does not matter.
* Do n-ary DT gives better Interpretability?

Yes. For the reasons mentioned earlier.

## Implementations

### Refactoring

Static and dynamic code analyses are performed during source code reviews of gosdt and mgosdt. Static code analysis is done during the development of our codebase; Then we perform dynamic code analysis in studying how the code behaves during execution.

### Static Code Analysis

### Dynamic Code Analysis 

- **Thread, address sanitizer**. We debug our codes with  address sanitizer with `fsanitize` flag. There were large number of memory leaks, data races and unitialized memory issues on the original GOSDT codebase.
- Valgrind
- **CPU Profiler**. We run Linux profiler `perf` combine with flame chart to get a better overview of possible red spots might cause performance slow down. Nothing been detected. See [cpu-profiling](https://gitlab.com/leannejdong/mgosdt/-/blob/main/cpu-profiling.md)

In [1]:
from sklearn import datasets
import xgboost as xgb
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [2]:
from platform import python_version
print(python_version())

3.9.2


Let’s get all of our data set up. We’ll start off by creating a train-test split so we can see just how well XGBoost performs. We’ll go with an 80%-20% split this time.

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [4]:
#In order for XGBoost to be able to use our data, we’ll need to transform it into a specific format that XGBoost can handle. That format is called DMatrix. 
#It’s a very simple one-linear to transform a numpy array of data to DMatrix format:

D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [5]:
# Define xgboost
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

In [6]:
#Training and Testing
#We can finally train our model similar to how we do so with Scikit Learn:
model = xgb.train(param, D_train, steps)

[23:18:08] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [7]:
#Let’s now run an evaluation. 
# Again the process is very similar to that of training models in Scikit Learn:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))


Precision = 0.8444444444444444
Recall = 0.8055555555555555
Accuracy = 0.8333333333333334


### Further Exploration with XGBoost
That just about sums up the basics of XGBoost. But there are some more cool features that’ll help you get the most out of your models.
The gamma parameter can also help with controlling overfitting. It specifies the minimum reduction in the loss required to make a further partition on a leaf node of the tree. I.e if creating a new node doesn’t reduce the loss by a certain amount, then we won’t create it at all.
The booster parameter allows you to set the type of model you will use when building the ensemble. The default is gbtree which builds an ensemble of decision trees. If your data isn’t too complicated, you can go with the faster and simpler gblinear option which builds an ensemble of linear models.
Setting the optimal hyperparameters of any ML model can be a challenge. So why not let Scikit Learn do it for you? We can combine Scikit Learn’s grid search with an XGBoost classifier quite easily:


In [ ]:
from sklearn.model_selection import GridSearchCV

clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss", cv=3)
grid.fit(X_train, Y_train)
model.dump_model('dump.raw.txt')

## Embedding MGOSDT in Python

Our goal is to being able to run [example.py](https://gitlab.com/leannejdong/mgosdt/-/blob/main/python/example.py)
which calls the `GOSDT` class in [gosdt.py](https://gitlab.com/leannejdong/mgosdt/-/blob/main/python/model/gosdt.py).
This GOSDT class uses the C++ extension module `gosdt` defined in `python_extension.cpp`.
The implementations are inherantly done in C++ by creating a `GOSDT` object called `model`, which perform operations `fit`,  `predict`, `score`.

```python
import pandas as pd
import numpy as np
from model.gosdt import GOSDT

dataframe = pd.DataFrame(pd.read_csv("/home/leanne/Dev/mgosdt/experiments/datasets/iris/data.csv"))

X = dataframe[dataframe.columns[:-1]]
y = dataframe[dataframe.columns[-1:]]

hyperparameters = {
    "regularization": 0.04,
    "time_limit": 3600,
    "verbose": True
}

model = GOSDT(hyperparameters)
model.fit(X, y)
# model.load("python/model/model.json")
# model.load("../gosdt_icml/model.json")
print("Execution Time: {}".format(model.time))

prediction = model.predict(X)
training_accuracy = model.score(X, y)
print("Training Accuracy: {}".format(training_accuracy))
print("Size: {}".format(model.leaves()))
print("Loss: {}".format(1 - training_accuracy))
print("Risk: {}".format(
    model.leaves() * hyperparameters["regularization"]
    + 1 - training_accuracy))
model.tree.__initialize_training_loss__(X, y)
print(model.tree)
print(model.latex())
```

It is simple C! First we need a function take some python object as input. We need to parse them and get some native variables in C++. In this case, we struct some char pointer that store the python objects. Then we do our operations in C++, configuring algorithm. Then do the step back from my native variables to python objects. Next, register the function within a module's symbol table. It is a table in which declared which functions are being supported by the module. (Remember, all Python functions live in a module, even if they are actually C functions) The name `add` would be the python name we use to call this module.
Finally, we have to declare the initialization of the module. It is the function will be called when importing the module
### Python C API
```cpp
#include <Python.h>
#include <iostream>
#include <fstream>
#include <sstream>
#include <string>
#include "gosdt.hpp"
// @param args: contains a single string object which is a JSON string containing the algorithm configuration
static PyObject * configure(PyObject * self, PyObject * args) {
    const char * configuration;
    if (!PyArg_ParseTuple(args, "s", & configuration)) { return NULL; }

    std::istringstream config_stream(configuration);
    GOSDT::configure(config_stream);

    return Py_BuildValue("");
}

// @param args: contains a single string object which contains the training data in CSV form
// @returns a string object containing a JSON array of all resulting models
static PyObject * fit(PyObject * self, PyObject * args) {
    const char * dataset;
    if (!PyArg_ParseTuple(args, "s", & dataset)) { return NULL; }

    std::istringstream data_stream(dataset);
    GOSDT model;
    std::string result;
    model.fit(data_stream, result);

    return Py_BuildValue("s", result.c_str());
}

// @returns the number of seconds spent training
static PyObject * time(PyObject * self, PyObject * args) { return Py_BuildValue("f", GOSDT::time); }

// @returns the number of iterations spent training
static PyObject * iterations(PyObject * self, PyObject * args) { return Py_BuildValue("i", GOSDT::iterations); }

// @returns the number of vertices in the depency graph
static PyObject * size(PyObject * self, PyObject * args) { return Py_BuildValue("i", GOSDT::size); }

// @returns the current status code
static PyObject * status(PyObject * self, PyObject * args) { return Py_BuildValue("i", GOSDT::status); }

// Define the list of methods Python intepreter needs to know about for a module
static PyMethodDef gosdt_methods[] = {
    // { method name, method pointer, method parameter format, method description }
    {"configure", configure, METH_VARARGS, "Configures the algorithm using an input JSON string"},
    {"fit", fit, METH_VARARGS, "Trains the model using an input CSV string"},
    {"time", time, METH_NOARGS, "Number of seconds spent training"},
    {"iterations", iterations, METH_NOARGS, "Number of iterations spent training"},
    {"size", size, METH_NOARGS, "Number of vertices in the depency graph"},
    {"status", status, METH_NOARGS, "Check the status code of the algorithm"},
    {NULL, NULL, 0, NULL}
};

// Define the module
static struct PyModuleDef gosdt = {
    PyModuleDef_HEAD_INIT,
    "gosdt", // Module Name
    "Generalized Optimal Sparse Decision Tree", // Module Description
    -1, // Size of per-interpreter state
    gosdt_methods // Module methods
};

// Initialize the module
PyMODINIT_FUNC PyInit_gosdt(void) {
    return PyModule_Create(&gosdt);
}
```

## Importing C++ extension to the Python Library

In [3]:
%cd

/home/leanne


In [4]:
%cd Dev/mgosdt

/home/leanne/Dev/mgosdt


In [5]:
import gosdt


In [5]:
%cd ../python

/home/leanne/Dev/mgosdt/python


In [6]:
import pandas as pd
import numpy as np
from model.gosdt import GOSDT

In [11]:
import gosdt

with open ("/home/leanne/Dev/mgosdt/experiments/datasets/tennis/tennis.csv", "r") as data_file:
    data = data_file.read()

with open ("/home/leanne/Dev/mgosdt/experiments/configurations/debug.json", "r") as config_file:
    config = config_file.read()


print("Config:", config)
print("Data:", data)

gosdt.configure(config)
result = gosdt.fit(data)

print("Result: ", result)
print("Time (seconds): ", gosdt.time())
print("Iterations: ", gosdt.iterations())
print("Graph Size: ", gosdt.size())


Config: {
    "uncertainty_tolerance": 0.0,
    "regularization": 0.01,
    "upperbound": 0.0,
    
    "time_limit": 86400,
    "worker_limit": 1,
    "tile_limit": 1,
    "model_limit": 1,

    "verbose": true,
    "diagnostics": false,
    "balance": true,
    "non_binary": true,

    "look_ahead": true,
    "similar_support": false,
    "greedy_sampling": false,
    "cancellation": true,
    "indifference": false,
    "strong_indifference": false
}

Data: outlook,temp,humidity,wind,play
Sunny,Hot,High,Weak,No
Sunny,Hot,High,Strong,No
Overcast,Hot,High,Weak,Yes
Rain,Mild,High,Weak,Yes
Rain,Cool,Normal,Weak,Yes
Rain,Cool,Normal,Strong,No
Overcast,Cool,Normal,Strong,Yes
Sunny,Mild,High,Weak,No
Sunny,Cool,Normal,Weak,Yes
Rain,Mild,Normal,Weak,Yes
Sunny,Mild,Normal,Strong,Yes
Overcast,Mild,High,Strong,Yes
Overcast,Hot,Normal,Weak,Yes
Rain,Mild,High,Strong,No

Result:  [
  {
    "children": [
      {
        "in": "Overcast",
        "then": {
          "complexity": 0.009999999776482582

In [7]:
dataframe = pd.DataFrame(pd.read_csv("/home/leanne/Dev/mgosdt/experiments/datasets/iris/data.csv"))

X = dataframe[dataframe.columns[:-1]]
y = dataframe[dataframe.columns[-1:]]

hyperparameters = {
    "regularization": 0.04,
    "time_limit": 3600,
    "verbose": True
}

model = GOSDT(hyperparameters)
model.fit(X, y)
print("Execution Time: {}".format(model.time))

Execution Time: 11.60099983215332


In [9]:
#dataframe = pd.DataFrame(pd.read_csv("/home/leanne/Dev/mgosdt/experiments/datasets/monk_2/data.csv"))
dataframe = pd.DataFrame(pd.read_csv("/home/leanne/Dev/mgosdt/experiments/datasets/tennis/tennis.csv"))
X = dataframe[dataframe.columns[:-1]]
y = dataframe[dataframe.columns[-1:]]

hyperparameters = {
    "regularization": 0.1,
    "time_limit": 3600,
    "verbose": True,
}

model = GOSDT(hyperparameters)
model.fit(X, y)
print("Execution Time: {}".format(model.time))

Execution Time: 0.0
